# Lecture1.introduction
> [这里是讲义](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture1.pdf)

图像被称作互联网的“暗物质”  
David Marr，1970s，stages of Visual Representation  
1. input image
2. primal sketch(edge image)
3. 2 1/2-D sketch 
4. 3-D model  

Face Detection,2001  
Histogam of Gradients(HoG),Dalal & Triggs,2005  
PASCAL Visual Object Challenge(20 object categories)  

_现代图像识别问题是特征多，维度高，用算法经常过拟合_  

*Image Net Challenge所用算法演变*  
- Lin CVPR 2011---svm  
- Krizhevsky NIPS 2012------cNN,Supervision(AlexNet)  
- Szegedy arxiv 2014/Simonyan arxiv 2014-------VGG GoogleNet  
- Microsoft Research Asia 2015-----152 layer Residual Networks  
***

# Lecture2.image classificatioon
>[这里是讲义](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture2.pdf)

## Data-Driven Approach
1. collect a dataset of images and labels
2. use machine learning to train a classifier
3. evaluate the classifier on new images  

## 机器学习方法用于预测分类，一般分两个函数（步骤）：
1. 输入函数→train
2. 输出函数→predict  

## 常用分类器1：Nearest Neighbor
1. memorize all data and labels
2. predict the label of the most similar train image  

*Distance Metric to compare images*  
L1 distance(Manhattan distance):$$d_1(I_1,I_2)=\sum_{P}\left | I_1^P - I_2^P \right |$$  

训练时间复杂度O(1), 预测时间复杂度O(N)，但是我们的需求是，训练时间可以长，但是预测速度越快越好  
_缺点_：不准确,噪声点误分类  
_改进_：K-Nearest Neighbor，给定一个K，将最邻近的K个样本点的分类作为最终预测结果

*用欧几里得距离作为Distance Meric*  
L2(Euclidean)distance:$$d_2(I_1,I_2)= \sqrt{\sum_{P}(I_1^P - I_2^P)^{2}}$$  
### L1与L2区别 
L1依赖于所选择的坐标系，若旋转坐标系，L1距离会变化  
L2不依赖于坐标系  
如果输入特征有特别含义，则用L1较好，如果特征间无差别，则用L2较好 

### 设置K近邻的超参数  
1. ×选择最佳超参数K（BAD：K=1总是对训练集拟合最好）
2. ×分为训练集和测试集（BAD：只在测试集上预测效果好，不知道未知数据预测效果如何）  
3. √分为训练集、验证集和测试集
4. √cross-validation（在小数据集中非常有用，但在deep learning中不常用）

_K-Nearest Neighbor on images **never used**_
- Very slow at test time
- Distance metrics on pixels are not informative
L2距离对样本数据变化（图像变化，如遮挡，变换）不敏感
- curse of dimensionality
随着维度增加，数据个数（采样点）指数级增多  

### summary
- In image classification we start with a training set of images and labels , andd must predict labels on the test set 
- The K-Neatest Neighbors classifier predicts labels based on nearest training examples
- Distance metric and K are hyperparameters
- Choose hyperparameters using the validation set; only run on the test set once at the very end!  

## 常用分类器2：线性分类
### Parametric Approach
一个尺寸为32（pixels）×32（pixels）×3（RGB）的图片，转为含3072数字的Array，通过权重矩阵W，转换为10个给定分类的分值  
$$f(x) = Wx + b$$

*不需要测试集*

该方法试图在高维空间用线性划分分类,但对线性不可分集无用  

如何用cost function评价W的好坏，下节课讲  
***

# Lecture3.Loss Functions and Optimization

>[这里是讲义](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture3.pdf)

## 评价权重矩阵W的方法
1. 定义一个loss function量化分类的好坏
2. 找出最小化以上函数的参数（optimization）  

## Loss function
### 一般表示：  
假设数据集的样本表示为$\left\{(x_i,y_i)\right\}^{N}_{i=1}$  
$x_i$为图像，$y_i$为label（int）  
数据集的损失表示为：$$L = \frac{1}{N} \sum_{i} L_i(f(x_i,W),y_i)$$  

### Multiclass SVM loss(Hinge Loss)：  
$$L_i = \sum_{j\neq y_i}\max\left ( 0,s_j - s_{y_i} + 1 \right )$$
其中：$s=f(x_i,W)$($s_j$表示预测分类分数，$s_{y_i}$表示其他分类分数)  
$so:$  
$$L = \frac{1}{N} \sum_{i=1}^{N} L_i$$
几点特性：  
- Hinge损失不关心具体数值，只关心不同值之间的大小关系
- Hinge损失的取值范围为$[0,-\infty)$
- 若初始权重矩阵得出的s≈0，去掉一个分类重新赋值计算
- 若想降低算法对误分类的容忍度，可用平方hinge损失  

#### Hinge损失函数代码
```python
def L_i_wectorized(x, y, W):
    scores = W.dot(x)
    margins = np.maximum(0, scores - score[y] + 1)
    margins[y] = 0
    loss_i = np.sum(margins)
    return loss_i
```
margins[y] = 0可保证迭代时跳过待计算分类（$s_j$），实现$j\neq y_i$  

### 正则项
$$L = \frac{1}{N} \sum_{i=1}^{N}\sum_{j\neq y_i} \max(0,f(x_i;W)_j - f(x_i;W)_{y_i} +1)+\lambda R(W)$$  
#### 常用正则项
- L2 regularization $R(W)=\sum_k \sum_l W_{k,l}^2$ 
- L1 regularization $R(W)=\sum_k \sum_l \left|W_{k,l}\right|$
- Elastic net(L1+L2) $R(W)=\sum_k \sum_l \beta W_{k,l}^2 +\left|W_{k,l}\right|$
- Max norm regularization
- Dropout
- Batch normalization
- stochastic depth  

### Softmax Classifier(Multinomial Logistic Regression)
Softmax函数实际为normalize化的指数函数，即$softmax(x)=normalize(e^x)$  
分类器损失函数如下：  
$$P(Y=k|X=x_i)=\frac{e^{s_k}}{\sum_j e^{s_j}}\quad\text{其中}s=f(x_i;W)$$
$$L_i=-logP(Y=y_i|X=x_i)$$
$$so:L_i=-log(\frac{e^{s_{y_i}}}{\sum_j e^{s_j}})$$

## Optimization
### Follow the slope
- In 1-dimension,the slope is the derivative of a function
- In multiple dimensions, the slope is the gradient  

### Analytic Gradient
fast,efficient,but error-pron,could be debugging through numerical gradient（which is slow, approximat, easy to write）  
### Gradient Descent
```python
while True:
    weights_grad = evaluate_gradient(loss_fun, data, weights)
    weights += - step_size * weights_grad # perform parameter update
```
### Stochastic Gradient Descent(SGD)
Full sum expensive when N is large  
Approximate sum using a **minibatch** of examples(32/64/128 common size)   
```python
while True:
    data_batch = sample_training_data(data, 256) # sample 256 examples
    weights_grad = evaluate_gradient(loss_fun, data_batch, weights)
    weights += - step_size * weights_grad # perform parameter update  
```
### Linear Classification Loss Visualization
用动画显示训练过程，可调超参数，观察收敛速度和方式
[链接](http://vision.stanford.edu/teaching/cs231n-demos/linear-classify/)  
### 对于图片特征线性不可分
可采用以下方法进行特征变换：
1. Color Histogram  
把每个色彩的pixel数累计在相应的color bar下  
2. Histogram of Oriented Gradients(HoG)  
3. Bag of Words  
 - Step1.Build codebook
 - Step2.Encode images  
 ***

# Lecture4.Introduction to Neural Networks
>[这里是讲义](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture4.pdf)

## BP算法相关
### Chain rule
Backpropagation中的重要方法，利用chain rule由输出节点反向计算输入节点的梯度。  
假设三层神经网络（一输入层，两隐藏层），输入矩阵为$X = [x_1,x_2,...,x_m]$，分别经过Hidden layer1（权重矩阵$\theta_1$，激活函数$f(z)$）,Hidden layer2（权重矩阵$\theta_2$，激活函数$g(z)$），到达输出层（权重矩阵$\theta_3$，激活函数$h(z)$），输出向量为$y = [y_1,y_2,...,y_n]。  

即：  

$$Z^{(1)} = \theta_1\cdot X$$  

$$Z^{(2)} = \theta_2\cdot f(Z^{(1)})$$  

$$Z^{(3)} = \theta_3\cdot g(Z^{(2)})$$  

$$y = h(Z^{(3)})$$


那么，在算得输出节点损失$\delta $后，可根据chain rule反向传播到Hidden layer2的损失如下：  

$$\frac{\partial y}{\partial Z_2} = $$


### forward
由输入向前计算得到输出，保存中间变量（$Z^{(i)}$）
### backward
应用chain rule及中间变量，计算各步关于输入量的损失

## Neural Network
### 几种激活函数
- Sigmoid  
$$\sigma(x) = \frac{1}{1+e^{-x}}$$
- tanh $$tanh(x)$$
- ReLU (Rectified Linear Units)[Yann LeCun,2009](http://yann.lecun.com/exdb/publis/pdf/jarrett-iccv-09.pdf) $$\max(0,x)$$
- Leaky ReLU $$\max(0.1x,x)$$
- ELU  
$$\left\{
\begin{aligned}
x\quad x\geqslant 0 \\
\alpha (e^{x} -1)\quad x<0
\end{aligned}
\right.$$  
***

# Lecture5.Convolutional Neural Networks
>[这里是讲义](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture5.pdf)  

## History
- Frank Rosenblatt, 1957, Perceptron
- Widrow and Hoff, 1960, Adaline/Madaline
- Rumelhar,1986, First time back-propagation became popular
- Hinton and Salakhutdinov, 2006, [Reinvigorated research in Deep Learning](https://www.cs.toronto.edu/~hinton/science.pdf)
- Hinton Krizhevsky and Sutskever, 2012, [First strong results:AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)  

## today
- Hinton, 2012, [Reproduced with permission](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)
- Ren He and Girshick, 2015, [Faster R-CNN](https://papers.nips.cc/paper/5638-faster-r-cnn-towards-real-time-object-detection-with-region-proposal-networks.pdf)
- Taigman, 2014, [face recognition](https://www.cs.toronto.edu/~ranzato/publications/taigman_cvpr14.pdf)  

## Convolution
卷积的定义：[百度百科](https://baike.baidu.com/item/%E5%8D%B7%E7%A7%AF/9411006?fr=aladdin&fromid=18080681&fromtitle=Convolution)
[wikipedia](https://en.wikipedia.org/wiki/Convolution)  
filter 和 image的元素相乘求和：  
$$f[x,y]*g[x,y] = \sum_{n_1=-\infty}^{\infty}\sum_{n_2=-\infty}^{\infty} f[n_1,n_2]\cdot g[x-n_1,y-n_2]$$  

## Filter and padding
1. 用3×3的filter卷过7×7像素的图片  
    - stride=1 得到一个5×5的output
    - stride=2 得到一个3×3的output
    - stride=3 不匹配！
    - 一般说来，卷积层尺寸为（N-F）/stride+1
2. 用3×3边缘用0填充的filter卷过7×7像素（边缘同样用0填充）的图片
    - stride=1 得到一个7×7的output
    - stride=3 得到一个3×3的output
    - 若想得到同样尺寸的卷积层：
        - filter尺寸为3×3时，需填充（zero pad）1行像素0
        - filter尺寸为5×5时，zero pad with 2
        - filter尺寸为7×7时，zero pad with 3  
        - 一般说来zero pad with（F-1）/2即可得到同样尺寸的卷积层（F为filter尺寸）  
        
3. 1中的padding称为‘valid padding’，2中的padding称为‘same padding’（不会丢掉edge和Corner的信息）  

## Pooling layer
- makes the representations smaller and more manageable
- operates over each activation map independently  

### max polling  

### summary
- ConvNets 是卷积层（CONV），池化层（POOL），全链接层（FC）的堆叠
- 倾向于用更小的filter和更深的结构
- 倾向于用更多的卷积层、更少的池化层和全链接层
- 典型结构：[(CONV-RELU)×N-POOL]×M-(FC-RELU)×K,SOFTMAX
    - N一般5以内，M很大，K一般2以内  
***    

# Lecture6.Training Neural Networks1
>[这里是讲义](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture6.pdf)

## Mini-batch SGD
循环：
- 随机取一部分（batch）样本
- 前向传播得到损失
- Backpropagation 计算梯度
- 用梯度更新参数  

## Activation Functions  

常见激活函数：  

### sigmoid
- Squashes numbers to range[0,1]
- 曾经最经典的激活函数，良好的解释性，有很好的性质（指数、导数），但不适用于Backpropagation  
缺点：  
- 使梯度‘消失’，Backpropagation中（梯度为$\frac{\partial{f}}{\partial{x}}=sigmoid(1-sigmoid)$）：
    - 当x为较小负数，梯度趋近于0
    - 当x为0（附近）时，梯度可继续计算
    - 当x为较大正数，梯度趋近于0  
- Sigmoid的输出不是以0为中心
    - 若输入为全正或全负，
- 指数计算耗时长  

### tanh(x)
- Squashes numbers to range[-1,1]
- 输出以0为中心（优于sigmoid）
- 仍使梯度在较大正负值处消失  

### ReLU
- 在正数区域不会‘饱和’
- 计算快
- 收敛速度快于$tanh(x)$和$sigmoid$ （一般快6倍）  
缺点：
- 输出不以0为中心
- 在负数区依然会饱和，且梯度为0
- 当数据位于非激活区时（负数），该部分权重也得不到更新
    - 解决办法：初始化ReLU时，加一个小的正偏置（bias，比如0.01）  
    
### Leaky ReLU
- 不会‘饱和’
- 计算快
- 收敛速度快于$tanh(x)$和$sigmoid$ （一般快6倍）
- 不存在像ReLU一样的缺点  
- Parametric Rectifier(PReLU):$f(x)=\max{(\alpha x,x)}$  

### ELU
- 具有ReLU的优点
- 输出均值接近0
- 相比Leaky ReLU，负饱和区对噪声的鲁棒性更好  

### Maxout 'Neuron'[Goodfellow,2013](http://proceedings.mlr.press/v28/goodfellow13.pdf)  

### 建议
- 首选ReLU，慎重选择学习率
- 尝试Leaky ReLU、Maxout、ELU
- 尝试tanh（可能并不会太好）
- **不要用sigmoid！！！！！**  

## Data Preprocessing
### preprocess the data
- zero-mean
```python
X -= np.mean(X, axis=0)
```
- normalized（机器学习中常用方法，但图像处理中一般不需要）
```python
X /= np.std(X, axis=0)
```
- PCA/Whitening 与normalized一样，不常用于图像处理  

### 例子
比如CIFAR-10中的32×32×3的图像，有以下两种处理方式：
- 减去整个图像（所有样本）的平均（AlexNet）  
（mean image是一个32×32×3的array）  
- 减去每个通道的平均（VGGNet）  

## Weight initialization
- small random numbers（服从高斯分布，均值为0，方差为0.01）  
```
W = 0.01 * np.random.randn(D,H)
```  
    - 小型神经网络可以，深度网络不合适（方差衰减，所有值变为0）
    - 一个比较好的方法[‘Xavier initialization’](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.207.2059&rep=rep1&type=pdf)
    - 另一个方法[He et al. 2015](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf)
    
## Batch Normalization
步骤：  
1. 在各维度上独立的求出均值和方差
2. Normalize
3. （一般）插入全链接层和卷积层之后
优点：  
- improve gradient flow through the network
- 允许更高的学习率 
- 降低对（权重？）初始化的依赖  
注意：
- 在测试集上不再重新计算mean

## 训练模型中出现的几点问题
1. loss不下降或下降很慢（但准确率在上升）
    - 学习率太低。由于softmax选择最大值的特性，可能造成准确率上升而损失不下降的现象
2. loss返回NaN（explode）
    - 学习率太高
3. 学习率一般取1e-3到1e-5  

## Hyperparameter Optimization
### Cross-validation
1. 用较小的迭代次数粗略的选出大概合适的超参数范围
2. 增多迭代次数，继续选出精确的超参数
    - 如果cost一直大于3倍于初始cost，说明参数错误，尽早终止  
***

# Lecture7.Training Neural Networks2
>[这里是讲义](